Datasets used: 
- `adult (Adult)` :<br>The UCI Adult dataset contains 14 features, which can be divided into two categories: demographic and income-related.<br> The demographic features include:

    age: continuous.<br>
    fnlwgt : continuous, represents final weight, which is the number of units in the target population that the responding unit represents.<br>
    workclass: categorical, with values 'Private', 'Local-gov', 'Self-emp-not-inc', 'Federal-gov', 'State-gov', 'Self-emp-inc', 'Without-pay'.<br>
    education: categorical, with values '11th', 'HS-grad', 'Assoc-acdm', 'Some-college', '10th', ..., '9th', 'Doctorate', '12th', '1st-4th', 'Preschool'.<br>
    education-num: continuous with values 1 to 16, one number assigned to each label of education feature.<br>
    marital-status: categorical, with values 'Never-married', 'Married-civ-spouse', 'Widowed', 'Separated', 'Divorced', 'Married-spouse-absent', 'Married-AF-spouse'.<br>
    occupation: categorical, with values such as 'Machine-op-inspct', 'Farming-fishing', 'Protective-serv'.<br>
    relationship: categorical, with values 'Own-child', 'Husband', 'Not-in-family', 'Unmarried', 'Wife', 'Other-relative'.<br>
    race: categorical, with values such as white, black, and Asian.<br>
    sex: categorical, with values male and female.<br>
    
    The income-related features include:<br>

    hours-per-week: continuous.<br>
    native-country: categorical, with values such as United-States, Mexico, and Germany.<br>
    capital-gain: continuous, represent the amount of money an individual has gained from the sale of investments such as stocks, bonds, or real estate.<br>
    capital-loss: continuous, represent the amount of money an individual has lost from the sale of investments such as stocks, bonds, or real estate..<br>
    The target feature is the income, which is binary:<br>
    income: categorical, with values less than or equal to 50K and greater than 50K.<br>

Reference Links: https://archive.ics.uci.edu/ml/datasets/adult

In [1]:
import pandas as pd
import numpy as np

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import warnings
warnings.filterwarnings('ignore')
import facts
from facts.clean import clean_dataset
from facts import valid_ifthens_with_coverage_correctness, rules2rulesbyif
from facts.models import customLogisticRegression
from facts.parameters import ParameterProxy
from facts.formatting import recourse_report_reverse, print_recourse_report, print_recourse_report_cumulative, print_recourse_report_KStest_cumulative
from facts.utils import load_rules_by_if
from facts.fairness_metrics_aggr import get_diff_table, get_comb_df, get_analysis_dfs

In [2]:
cor_thres = 0.5
cost_budget = 10
top_count = 20
c_inf = 5

# Dataset

In [3]:
urlfile= "https://raw.githubusercontent.com/columbia/fairtest/master/data/adult/adult.csv"

X = pd.read_csv(urlfile)
df = clean_dataset(X, "adult")

age = [val.left for val in df.age.unique()]
age.sort()

df.head()

,age,Workclass,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,"(34.0, 41.0]",State-gov,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,FullTime,United-States,0
1,"(41.0, 50.0]",Self-emp-not-inc,13,Married-civ-spouse,Exec-managerial,Married,White,Male,0,0,PartTime,United-States,0
2,"(34.0, 41.0]",Private,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,FullTime,United-States,0
3,"(50.0, 90.0]",Private,7,Married-civ-spouse,Handlers-cleaners,Married,Black,Male,0,0,FullTime,United-States,0
4,"(26.0, 34.0]",Private,13,Married-civ-spouse,Prof-specialty,Married,Black,Female,0,0,FullTime,Cuba,0


In [4]:
y = df['income']
X = df.drop('income', axis=1)

num_features = X._get_numeric_data().columns.to_list()
cate_features = X.select_dtypes(include=['object','category']).columns.to_list()
ord_features = ['hours-per-week']

# Rules

In [5]:
rules_with_atomic_correctness = load_rules_by_if("rulesAdultFairTest_preproc.data")
rules_with_cumulative_correctness = load_rules_by_if("rulesAdultFairTest_cumulative.data")

In [6]:
feature_weights = {"race" : 100 , "sex": 100, "marital-status": 5, "relationship":5, "age": 10, "occupation": 4, "Workclass": 2, "native-country": 4,
                    "hours-per-week":2, "capital-gain": 1, "capital-loss":1, "education-num": 3}
features_with_binary_cost = cate_features
features_with_proportional_cost = num_features


comparators = facts.feature_change_builder(
    X,
    num_cols=features_with_proportional_cost,
    cate_cols=features_with_binary_cost,
    ord_cols=ord_features,
    feature_weights=feature_weights,
    num_normalization=True,
    feats_to_normalize = ["capital-gain","capital-loss"]
)
params = ParameterProxy(featureChanges=comparators)

In [7]:
facts.update_costs_cumulative(rules_with_cumulative_correctness, params)

# Macro Viewpoint

## Equal Cost of Effectiveness (Macro)

In [8]:
top_rules, subgroup_costs = facts.select_rules_subset(
    rules_with_atomic_correctness,
    metric="min-above-thr",
    sort_strategy="generic-sorting-ignore-exists-subgroup-empty",
    top_count=top_count,
    cor_threshold=cor_thres,
    filter_sequence = [
        # "remove-contained",
        # "remove-below-thr",        
        # "remove-fair-rules",
        # "keep-only-min-change"
    ],
    params=params
)

print_recourse_report(
    top_rules,
    subgroup_costs=subgroup_costs,
    show_subgroup_costs=True
)

If age = (34.0, 41.0], capital-loss = 0, marital-status =  Never-married, relationship =  Not-in-family:
	Protected Subgroup ' Male', 2.28% covered
		Make age = (41.0, 50.0], marital-status =  Married-civ-spouse, relationship =  Married with effectiveness 48.37%.
		Make age = (50.0, 90.0], marital-status =  Married-civ-spouse, relationship =  Married with effectiveness 43.79%.
		Make marital-status =  Married-civ-spouse, relationship =  Married with effectiveness 41.83%.
		Make age = (41.0, 50.0], capital-loss = 1902, marital-status =  Married-civ-spouse, relationship =  Married with effectiveness 84.97%.
		Aggregate cost of the above recourses = 20.44
	Protected Subgroup ' Female', 2.77% covered
		Make age = (41.0, 50.0], marital-status =  Married-civ-spouse, relationship =  Married with effectiveness 57.39%.
		Make age = (50.0, 90.0], marital-status =  Married-civ-spouse, relationship =  Married with effectiveness 53.04%.
		Make marital-status =  Married-civ-spouse, relationship =  M

## Equal Choice for Recourse

In [9]:
top_rules, subgroup_costs = facts.select_rules_subset(
    rules_with_atomic_correctness,
    metric="num-above-thr",
    sort_strategy="generic-sorting-ignore-forall-subgroups-empty",
    top_count=top_count,
    cor_threshold=cor_thres,
    filter_sequence = [
        # "remove-contained",
        # "remove-below-thr",
        # "remove-fair-rules"
    ],
    params=params
)

print_recourse_report(
    top_rules,
    subgroup_costs=subgroup_costs,
    show_subgroup_costs=True
)

If Workclass =  Private, age = (34.0, 41.0], occupation =  Sales:
	Protected Subgroup ' Male', 1.28% covered
		Make age = (41.0, 50.0], occupation =  Exec-managerial with effectiveness 61.63%.
		Make occupation =  Exec-managerial with effectiveness 51.16%.
		Make age = (41.0, 50.0], occupation =  Prof-specialty with effectiveness 60.47%.
		Make age = (50.0, 90.0], occupation =  Exec-managerial with effectiveness 51.16%.
		Make occupation =  Prof-specialty with effectiveness 45.35%.
		Make age = (50.0, 90.0], occupation =  Prof-specialty with effectiveness 51.16%.
		Make age = (41.0, 50.0] with effectiveness 44.19%.
		Make Workclass =  Local-gov, age = (41.0, 50.0], occupation =  Prof-specialty with effectiveness 51.16%.
		Make age = (50.0, 90.0] with effectiveness 20.93%.
		Make age = (41.0, 50.0], occupation =  Craft-repair with effectiveness 0.00%.
		Make occupation =  Craft-repair with effectiveness 0.00%.
		Make Workclass =  Self-emp-inc, age = (50.0, 90.0], occupation =  Exec-mana

# Micro Viewpoint

##  Equal Conditional Mean Recourse

In [10]:
top_rules, subgroup_costs = facts.select_rules_subset_cumulative(
    rules_with_cumulative_correctness,
    metric="fairness-of-mean-recourse-conditional",
    sort_strategy="generic-sorting-ignore-exists-subgroup-empty",
    top_count=top_count,
    filter_sequence = [
        # "remove-contained",
        # "remove-fair-rules"
    ],
    params=params
)

print_recourse_report_cumulative(
    top_rules,
    subgroup_costs=subgroup_costs,
    show_subgroup_costs=True,
    show_then_costs=True
)

If native-country =  United-States, race =  Black, relationship =  Not-in-family:
	Protected Subgroup ' Male', 2.27% covered
		Make relationship =  Married with effectiveness 2.65% and counterfactual cost = 5.
		Make race =  White with effectiveness 2.65% and counterfactual cost = 100.
		Make race =  White, relationship =  Married with effectiveness 3.97% and counterfactual cost = 105.
		Aggregate cost of the above recourses = 38.33
	Protected Subgroup ' Female', 4.02% covered
		Make relationship =  Married with effectiveness 0.00% and counterfactual cost = 5.
		Make race =  White with effectiveness 0.00% and counterfactual cost = 100.
		Make race =  White, relationship =  Married with effectiveness 0.72% and counterfactual cost = 105.
		Aggregate cost of the above recourses = 105.00
	Bias against  Female. Unfairness score = 66.667.
If age = (34.0, 41.0], race =  Black:
	Protected Subgroup ' Male', 1.36% covered
		Make age = (41.0, 50.0] with effectiveness 2.02% and counterfactual cost

## Equal Cost of Effectiveness (Micro)

In [11]:
top_rules, subgroup_costs = facts.select_rules_subset_cumulative(
    rules_with_cumulative_correctness,
    metric="min-above-corr",
    sort_strategy="generic-sorting-ignore-exists-subgroup-empty",
    top_count=top_count,
    cor_threshold = cor_thres,
    filter_sequence = [
        # "remove-contained",
        # "remove-fair-rules",
        # "keep-cheap-rules-above-thr-cor",
        # "keep-only-min-change",
        
    ],
    params=params
)

print_recourse_report_cumulative(
    top_rules,
    subgroup_costs=subgroup_costs,
    show_subgroup_costs=True,
    show_then_costs=True
)

If age = (34.0, 41.0], hours-per-week = FullTime, marital-status =  Never-married, native-country =  United-States, relationship =  Not-in-family:
	Protected Subgroup ' Male', 1.06% covered
		Make marital-status =  Married-civ-spouse, relationship =  Married with effectiveness 30.23% and counterfactual cost = 10.
		Make hours-per-week = OverTime, marital-status =  Married-civ-spouse, relationship =  Married with effectiveness 40.70% and counterfactual cost = 12.
		Make hours-per-week = BrainDrain, marital-status =  Married-civ-spouse, relationship =  Married with effectiveness 43.02% and counterfactual cost = 14.
		Make age = (50.0, 90.0], marital-status =  Married-civ-spouse, relationship =  Married with effectiveness 43.02% and counterfactual cost = 20.
		Make age = (41.0, 50.0], marital-status =  Married-civ-spouse, relationship =  Married with effectiveness 43.02% and counterfactual cost = 20.
		Make age = (50.0, 90.0], hours-per-week = MidTime, marital-status =  Married-civ-spouse

## Equal Effectiveness

In [12]:
top_rules, subgroup_costs = facts.select_rules_subset_cumulative(
    rules_with_cumulative_correctness,
    metric="total-correctness",
    sort_strategy="generic-sorting-ignore-forall-subgroups-empty",
    top_count=top_count,
    filter_sequence = [
        # "remove-contained",
        # "remove-fair-rules",
        # "keep-only-min-change"
    ],
    params=params
)

print_recourse_report_cumulative(
    top_rules,
    subgroup_costs=subgroup_costs,
    show_subgroup_costs=True,
    show_then_costs=True,
    correctness_metric=True
)

If Workclass =  Private, age = (34.0, 41.0], capital-loss = 0, native-country =  United-States, occupation =  Sales:
	Protected Subgroup ' Male', 1.19% covered
		Make occupation =  Craft-repair with effectiveness 0.00% and counterfactual cost = 4.0.
		Make occupation =  Prof-specialty with effectiveness 37.04% and counterfactual cost = 4.0.
		Make occupation =  Exec-managerial with effectiveness 40.74% and counterfactual cost = 4.0.
		Make Workclass =  Self-emp-inc, occupation =  Exec-managerial with effectiveness 56.79% and counterfactual cost = 6.0.
		Make age = (50.0, 90.0] with effectiveness 56.79% and counterfactual cost = 10.0.
		Make age = (41.0, 50.0] with effectiveness 56.79% and counterfactual cost = 10.0.
		Make age = (41.0, 50.0], occupation =  Craft-repair with effectiveness 56.79% and counterfactual cost = 14.0.
		Make age = (50.0, 90.0], occupation =  Exec-managerial with effectiveness 56.79% and counterfactual cost = 14.0.
		Make age = (50.0, 90.0], occupation =  Prof-s

## Fair Effectiveness-Cost Trade-Off

In [13]:
# taken from the other notebooks
affected_pop_sizes = {" Male": 6732, " Female": 4106}

top_rules,unfairness = facts.select_rules_subset_KStest(
    rules_with_cumulative_correctness,
    affected_pop_sizes,
    top_count=top_count
)

print_recourse_report_KStest_cumulative(
    top_rules,
    population_sizes=affected_pop_sizes,
    unfairness = unfairness,
    show_then_costs=True
    # show_cumulative_plots=True
)

If capital-gain = 0, capital-loss = 0:
	Protected Subgroup ' Male', 94.15% covered out of 6732
		Make capital-gain = 5013 with effectiveness 38.94% and counterfactual cost = 0.05.
		Make capital-gain = 5178 with effectiveness 39.87% and counterfactual cost = 0.05.
		Make capital-gain = 7298 with effectiveness 51.62% and counterfactual cost = 0.07.
		Make capital-gain = 7688 with effectiveness 53.23% and counterfactual cost = 0.08.
		Make capital-gain = 15024 with effectiveness 83.51% and counterfactual cost = 0.15.
		Make capital-loss = 1887 with effectiveness 83.51% and counterfactual cost = 0.43.
		Make capital-loss = 1902 with effectiveness 83.51% and counterfactual cost = 0.44.
		Make capital-loss = 1977 with effectiveness 83.51% and counterfactual cost = 0.45.
		Make capital-gain = 99999 with effectiveness 100.00% and counterfactual cost = 1.0.
	Protected Subgroup ' Female', 93.84% covered out of 4106
		Make capital-gain = 5013 with effectiveness 13.06% and counterfactual cost = 0

## Equal Effectiveness within Budget (Micro)

In [14]:
top_rules, subgroup_costs = facts.select_rules_subset_cumulative(
    rules_with_cumulative_correctness,
    metric="max-upto-cost",
    sort_strategy="generic-sorting-ignore-exists-subgroup-empty",
    top_count=top_count,
    cost_threshold = cost_budget,
    filter_sequence = [
        # "remove-contained",
        # "remove-fair-rules",
        # "remove-above-thr-cost"
        #"remove-below-thr",
        #"keep-only-min-change",
        
    ],
    params=params
)

print_recourse_report_cumulative(
    top_rules,
    subgroup_costs=subgroup_costs,
    show_subgroup_costs=True,
    show_then_costs=True,
    correctness_metric = True
)

If Workclass =  Private, age = (34.0, 41.0], capital-loss = 0, occupation =  Sales, race =  White:
	Protected Subgroup ' Male', 1.16% covered
		Make occupation =  Craft-repair with effectiveness 0.00% and counterfactual cost = 4.0.
		Make occupation =  Prof-specialty with effectiveness 40.26% and counterfactual cost = 4.0.
		Make occupation =  Exec-managerial with effectiveness 44.16% and counterfactual cost = 4.0.
		Make Workclass =  Self-emp-inc, occupation =  Exec-managerial with effectiveness 59.74% and counterfactual cost = 6.0.
		Make age = (50.0, 90.0] with effectiveness 59.74% and counterfactual cost = 10.0.
		Make age = (41.0, 50.0] with effectiveness 59.74% and counterfactual cost = 10.0.
		Make age = (41.0, 50.0], occupation =  Craft-repair with effectiveness 59.74% and counterfactual cost = 14.0.
		Make age = (50.0, 90.0], occupation =  Exec-managerial with effectiveness 59.74% and counterfactual cost = 14.0.
		Make age = (50.0, 90.0], occupation =  Prof-specialty with effe

## Get ranking of subgroups based on metrics

In [15]:
from facts.fairness_metrics_aggr import make_table, auto_budget_calculation

In [16]:
rules_with_both = {}
for ifc, all_thens in rules_with_cumulative_correctness.items():
    new_all_thens = {}
    for sg, (cov, thens_cum) in all_thens.items():
        thens_atom = rules_with_atomic_correctness[ifc][sg][1]
        thens_atom_dict = {then: atom_cor for then, atom_cor in thens_atom}
        new_all_thens[sg] = (cov, [(then, thens_atom_dict[then], cumcor) for then, cumcor, _cost in thens_cum])
    rules_with_both[ifc] = new_all_thens

In [17]:
budgets = auto_budget_calculation(rules_with_cumulative_correctness, cor_thres=0.5, percentiles=[0.3, 0.6, 0.9])

df = make_table(
    rules_with_both,
    sensitive_attribute_vals=[" Male", " Female"],
    effectiveness_thresholds=[0.3, 0.7],
    cost_budgets=budgets,
    params=params
)
# dropping 'weighted-average' and 'mean-cost-cinf'
df.drop(['weighted-average', 'mean-cost-cinf'], axis=1, inplace=True)

In [18]:
#table of differences between Male and Female for each metric
diff = get_diff_table(df, sensitive_attribute_vals=[" Male", " Female"], with_abs = True)
diff = diff.set_index('subgroup')
ranked = diff.drop(columns=[('Fair Effectiveness-Cost Trade-Off','bias')]).mask(diff == 0).rank(ascending = False,axis=0,method='dense')
ranked = ranked.replace(np.nan,"Fair")

In [20]:
rev_bias_metrics = ['Equal Effectiveness', 'Equal Effectiveness within Budget']
comb_df = get_comb_df(df, ranked, diff, rev_bias_metrics, sensitive_attribute_vals=[" Male", " Female"])
analysis_df,rank_analysis_df = get_analysis_dfs(comb_df,ranked, sensitive_attribute_vals=["Male", "Female"])

In [21]:
comb_df.head()

(Equal Cost of Effectiveness(Macro), 0.3)  \
                                                                                       rank   
subgroup                                                                                      
capital-loss = 0                                                                        1.0   
capital-gain = 0                                                                       22.0   
capital-gain = 0, capital-loss = 0                                                     22.0   
capital-loss = 0, native-country =  United-States                                       1.0   
capital-gain = 0, native-country =  United-States                                      24.0   

                                                                          \
                                                      score bias against   
subgroup                                                                   
capital-loss = 0                                        inf       Female   
capital-gain = 0                                   0.100111       Female   
capital-gain = 0, capital-loss = 0                 0.100111       Female   
capital-loss = 0, native-country =  United-States       inf       Female   
capital-gain = 0, native-country =  United-States  0.098461       Female   

                                                  (Equal Cost of Effectiveness(Macro), 0.7)  \
                                                                                       rank   
subgroup                                                                                      
capital-loss = 0                                                                       Fair   
capital-gain = 0                                                                       Fair   
capital-gain = 0, capital-loss = 0                                                     Fair   
capital-loss = 0, native-country =  United-States                                      Fair   
capital-gain = 0, native-country =  United-States                                      Fair   

                                                                      \
                                                  score bias against   
subgroup                                                               
capital-loss = 0                                    0.0         Fair   
capital-gain = 0                                    0.0         Fair   
capital-gain = 0, capital-loss = 0                  0.0         Fair   
capital-loss = 0, native-country =  United-States   0.0         Fair   
capital-gain = 0, native-country =  United-States   0.0         Fair   

                                                  (Equal Choice for Recourse, 0.3)  \
                                                                              rank   
subgroup                                                                             
capital-loss = 0                                                              13.0   
capital-gain = 0                                                              10.0   
capital-gain = 0, capital-loss = 0                                            10.0   
capital-loss = 0, native-country =  United-States                             11.0   
capital-gain = 0, native-country =  United-States                             11.0   

                                                                      \
                                                  score bias against   
subgroup                                                               
capital-loss = 0                                    1.0       Female   
capital-gain = 0                                    4.0       Female   
capital-gain = 0, capital-loss = 0                  4.0       Female   
capital-loss = 0, native-country =  United-States   3.0       Female   
capital-gain = 0, native-country =  United-States   3.0       Female   

                                                  (Equal Choice for Recourse

In [22]:
analysis_df

,Rank = 1 Count,Male bias against Count,Female bias against Count
"(Equal Cost of Effectiveness(Macro), 0.3)",1673,910,1714
"(Equal Cost of Effectiveness(Micro), 0.3)",1523,620,1743
"(Equal Cost of Effectiveness(Micro), 0.7)",290,203,452
"(Equal Cost of Effectiveness(Macro), 0.7)",301,238,397
Equal(Conditional Mean Recourse),764,4657,6394
"(Equal Effectiveness within Budget, 5.076880768807688)",1,1745,4830
"(Equal Choice for Recourse, 0.3)",2,1286,2121
Equal Effectiveness,1,7236,3556
"(Equal Effectiveness within Budget, 18.0)",1,3158,6708
"(Equal Effectiveness within Budget, 10.0)",1,2836,6272


In [23]:
rank_analysis_df

,"(Equal Cost of Effectiveness(Micro), 0.3)","(Equal Cost of Effectiveness(Macro), 0.7)","(Equal Choice for Recourse, 0.7)","(Equal Effectiveness within Budget, 5.076880768807688)","(Equal Effectiveness within Budget, 10.0)",Fair Effectiveness-Cost Trade-Off,"(Equal Cost of Effectiveness(Macro), 0.3)",Equal(Conditional Mean Recourse),"(Equal Choice for Recourse, 0.3)","(Equal Cost of Effectiveness(Micro), 0.7)",Equal Effectiveness,"(Equal Effectiveness within Budget, 18.0)"
"(Equal Cost of Effectiveness(Micro), 0.3)",1.0,33.0,7.0,2766.3,2253.6,1738.4,13.3,3637.5,12.0,28.0,1380.6,1998.6
"(Equal Cost of Effectiveness(Macro), 0.7)",42.8,1.0,4.8,3688.1,3979.4,3442.3,35.5,3785.7,12.1,8.2,3631.4,3790.4
"(Equal Choice for Recourse, 0.7)",58.0,12.0,1.0,3234.7,6977.7,3818.0,56.0,3549.5,12.0,24.7,8267.0,7542.5
"(Equal Effectiveness within Budget, 5.076880768807688)",1.0,33.0,7.0,1.0,32.0,65.0,1.0,7875.0,12.0,28.0,57.0,56.0
"(Equal Effectiveness within Budget, 10.0)",1.0,33.0,7.0,12.0,1.0,1.0,1.0,371.0,3.0,28.0,3.0,3.0
Fair Effectiveness-Cost Trade-Off,1.0,33.0,7.0,12.0,1.0,1.0,1.0,371.0,3.0,28.0,3.0,3.0
"(Equal Cost of Effectiveness(Macro), 0.3)",17.4,33.0,7.0,3114.8,2937.0,2655.4,1.0,3471.9,11.7,28.0,2174.5,2656.2
Equal(Conditional Mean Recourse),57.8,33.0,7.0,5789.9,7508.7,9312.6,55.8,1.0,14.0,28.0,7345.6,7232.4
"(Equal Choice for Recourse, 0.3)",1.0,33.0,7.0,36.5,9.5,8.0,1.0,827.0,1.0,28.0,11.5,11.5
"(Equal Cost of Effectiveness(Micro), 0.7)",40.6,9.2,5.4,3805.2,3851.3,3316.7,34.2,3663.9,12.1,1.0,3407.9,3744.8
